In [ ]:
!pip install trimesh
!pip install triangle 
!pip install shapely

In [ ]:
import os
os.chdir("..")
from ui import location_picker
location_picker.get_location()


In [ ]:
import glob
import numpy as np 
import re

In [ ]:
import trimesh
import strm2stl.numpy2stl.numpy2stl as np3D
import cv2

from skimage import io
from skimage import filters
import matplotlib.pyplot as plt

from scipy import ndimage as ndi
import os

In [ ]:
from ui import location_picker

In [ ]:
import numpy as np
from PIL import Image
import re
import os

def parse_extent_from_filename(filename):
	match = re.search(r'n([-\d.]+)_s([-\d.]+)_w([-\d.]+)_e([-\d.]+)', filename)
	if match:
		n, s, w, e = map(float, match.groups())
		return (n, s, e, w)
	else:
		raise ValueError(f"Could not parse extent from {filename}")

def intersect_bbox(bbox1, bbox2):
	N1, S1, E1, W1 = bbox1
	N2, S2, E2, W2 = bbox2
	N = min(N1, N2)
	S = max(S1, S2)
	E = min(E1, E2)
	W = max(W1, W2)
	if N > S and E > W:
		return (N, S, E, W)
	return None

def crop_tile_np(image_array, tile_bbox, crop_bbox):
	tile_N, tile_S, tile_E, tile_W = tile_bbox
	crop_N, crop_S, crop_E, crop_W = crop_bbox

	height, width = image_array.shape
	lat_per_pixel = (tile_N - tile_S) / height
	lon_per_pixel = (tile_E - tile_W) / width

	y1 = int((tile_N - crop_N) / lat_per_pixel)
	y2 = int((tile_N - crop_S) / lat_per_pixel)
	x1 = int((crop_W - tile_W) / lon_per_pixel)
	x2 = int((crop_E - tile_W) / lon_per_pixel)

	return image_array[y1:y2, x1:x2]

def parse_extent_from_filename(filename):
	match = re.search(r'n([-\d.]+)_s([-\d.]+)_w([-\d.]+)_e([-\d.]+)', filename)
	if match:
		parts = match.groups()
		# Remove trailing dots and convert to float
		try:
			n, s, w, e = [float(p.rstrip('.')) for p in parts]
		except ValueError:
			raise ValueError(f"Could not convert parts to float: {parts}")
		return (n, s, e, w)
	else:
		raise ValueError(f"Could not parse extent from filename: {filename}")

def stitch_tiles_no_rasterio(tile_files, target_bbox):
	print("==== Stitching tiles without rasterio ====")
	print(f"Target bounding box: {target_bbox}")
	

	rows = {}

	for fn in tile_files:
		tile_bbox = parse_extent_from_filename(os.path.basename(fn))
		#print(f"Tile: {fn}\n Parsed extent: {tile_bbox}")

		intersection = intersect_bbox(tile_bbox, target_bbox)
		#print(f" Intersection: {intersection}")

		if not intersection:
			#print(" Skipping due to no intersection.")
			continue

		try:
			image_array = io.imread(fn)	
		except Exception as e:
			print(f"⚠️ Failed to open {fn}: {e}")
			continue

		cropped = crop_tile_np(image_array, tile_bbox, intersection)
		row_key = intersection[0]
		rows.setdefault(row_key, []).append((intersection[3], cropped))  # use W for sorting

	if not rows:
		print("==== No tiles matched ====")
		return None

	stitched_rows = []
	for N in sorted(rows.keys(), reverse=True):
		tiles = sorted(rows[N], key=lambda t: t[0])
		row = np.hstack([img for _, img in tiles])
		stitched_rows.append(row)

	final_image = np.vstack(stitched_rows)
	print("✅ Finished stitching without rasterio.")
	return final_image


In [ ]:
def proj_map_height(mat,NSEW):
    n,s,e,w = NSEW

    m1, n1 = mat.shape
    xv,yv = np.meshgrid(range(n1),range(m1))
    
    xv = ((xv/n1)-0.5) * (e-w)
    xv = np.deg2rad(xv)
        
    yv = ((1-yv/m1)-0.5) * (n-s)

    yv = np.deg2rad(yv)
    #zv = xv_c * yv_c
    
    zv = np.cos(xv) * np.cos(yv)  # * (12756) 
    
    zv = zv * m1/(n-s) * 180/np.pi
    
    #zv = zv - np.min(zv)
        
    return zv
def proj_map_geo_to_2D(mat,NSEW):
    
    lat = NSEW[[0,1]]
    lon = NSEW[[2,3]]

    m, n = mat.shape
    xv,yv = np.meshgrid(range(n),range(m))

    xc = (n-1)/2
    yc = (m-1)/2
    xv_c = (xv - xc).astype(int)
    yv_c = (yv - yc).astype(int)

    lat_v = np.linspace(lat[0],lat[1],m)
    lat_v = np.deg2rad(lat_v[:,None])
    xv_adj = xv_c * np.cos(lat_v )

    xv2 = (xv_adj + xc).astype(int)
    yv2 = (yv_c + yc).astype(int)

    mat_adj = mat*0.0 
    mat_adj[:] = np.nan
    mat_adj[yv2, xv2] = mat[yv, xv]
    
    y1,y2 = np.min(yv2),np.max(yv2)
    x1,x2 = np.min(xv2),np.max(xv2)
    
    mat_adj = mat_adj[y1:y2,x1:x2]
    return mat_adj  

In [ ]:
def mat2coor(limits, matsize, index):
  [x1,x2,y1,y2] = index

  xs = np.array([x1,x2])
  xs = xs / matsize[0]
  xs = (xs * limits[1]) + limits[0]


  ys = np.array([y1,y2])
  ys = ys / matsize[1]
  ys = (ys * (limits[3]-limits[2])) + (limits[2])

  print(xs,ys)

  coor = [xs[0], xs[1], ys[0], ys[1]]

  return coor


In [ ]:
def savefile(out_dir, name, im2 ):

	if im2.min()< 1:
		print("warning values less than zero")
		return 
	
	
	if not os.path.isdir(out_dir): return

	out_dir = out_dir + "/" + name 
	os.makedirs(out_dir,exist_ok=True)

	print("Saving Image")
	np.save(out_dir + "/" + name + ".npy", im2)
	
	tri = np3D.numpy2stl(im2)
	facets = np3D.triangles_to_facets(tri)
	
	print("Saving STL")
	np3D.writeSTL(facets, out_dir + "/" + name + ".stl")



In [ ]:
Ocean_root = "C:\\Users\\eac84\\Desktop\\Desktop\\Tasks\\srtm_tifs\\gebco_2020_geotiff\\"
tile_files = glob.glob(Ocean_root+"*.tif")


In [ ]:
%matplotlib qt

In [ ]:
Ocean_root = "C:\\Users\\eac84\\Desktop\\srtm_tifs\\gebco_2020_geotiff\\*"
Ocean_root = "C:\\Users\\eac84\\Desktop\\Desktop\\Tasks\\srtm_tifs\\gebco_2020_geotiff\\"
strm_fns = glob.glob(Ocean_root+"*.tif")
strm_fns 

## CARIBE

In [ ]:
nsew = 31,6,-54,-102

In [ ]:

imW = io.imread(strm_fns[4])
imE = io.imread(strm_fns[5])

offset = np.array([90,90,-90,-90])
imE = NSEW_crop_(imE, nsew, offset)

offset = np.array([90,90,-180,-180])
imW = NSEW_crop_(imW, nsew, offset)

im = np.hstack((imW,imE))

plt.imshow(im, cmap="jet")

In [ ]:
im2 = im.copy()
im2[im2>0] = im2[im2>0]+1000

sz = tuple((np.array(im2.shape)[[1,0]]*0.2).astype(np.int))
im2 = im2 * 0.2
im2 = cv2.resize(im2,sz)

im2 = im2 * 0.02

#im3 = proj_map_height(im2, np.array([n,s,e,w]))
im4 = im2 #+ np.round(im3,3)
im4 = proj_map_geo_to_2D(im4, np.array([n,s,e,w]))

#im4[np.isnan(im4)] = np.nanmin(im4)
im4 = im4[:,~np.any(np.isnan(im4),axis=0)]
im4 = im4 - np.min(im4) + 1
im4 = im4[::-1]

In [ ]:
plt.figure()
plt.imshow(im4[::-1],cmap="jet")

In [ ]:
tri = np3D.numpy2stl(im4)
facets = np3D.triangles_to_facets(tri)
np3D.writeSTL(facets, "Ocean_flat.stl")

In [ ]:
vertices, faces = np3D.vertices_to_index(tri)
mesh = trimesh.Trimesh(vertices,faces)
#mesh.show()

## COLOMBIA

In [ ]:
from strm2stl.osm2stl import get_boundries_osmnx

In [ ]:
from strm2stl.create import coor2im
from strm2stl.dem2stl import embed_lines
from skimage import morphology
from skimage.draw import line_aa, polygon2mask

In [ ]:
nsew = 14,-6,-64,-84
nsew = np.array(nsew)

In [ ]:
imN = io.imread(strm_fns[5])
offset = np.array([90,90,-90,-90])
imN = NSEW_crop_(imN, nsew, offset)

imS = io.imread(strm_fns[1])
offset = np.array([0,0,-90,-90])
imS = NSEW_crop_(imS, nsew, offset)
im = np.vstack((imN,imS))

plt.imshow(im, cmap="jet")

In [ ]:
x1 = 300 
x3 = 2000
im_ = im.copy()
im_[im_>x1] = im_[im_>x1] + (x3-x1)
bx = (im_<=x1) & (im_>0)
im_[bx] = im_[bx] * (x3/x1)

im_[im_<0] = im_[im_<0]*0.5
im_[im_>0] = im_[im_>0] + 10

In [ ]:
scale = 0.5
sz = tuple((np.array(im_.shape)[[1,0]]*scale).astype(int))
im_ = im_ * scale
im_ = cv2.resize(im_,sz)
im_ = im_ * 0.01

In [ ]:
sealine = im_>0
sealine = morphology.binary_dilation(sealine, morphology.disk(2)) & ~sealine
im_ = im_ - (sealine* im_.ptp()*0.01)

In [ ]:
bounds,bbox = get_boundries_osmnx('Colombia')
boundry = bounds[-2]

In [ ]:
nsew = np.array(nsew)
coorlims = (nsew[np.array([0,1])], nsew[np.array([2,3])])
imlims = ((0,im_.shape[1]), (im_.shape[0], 0))
r,c = coor2im(coorlims, imlims, boundry.pts.T)

im_lines = np.zeros(im_.shape, dtype=np.uint8)

for i in range(len(r)-1):
	rr, cc, val = line_aa(int(round(r[i])), int(round(c[i])), int(round(r[i+1])), int(round(c[i+1])))
	im_lines[rr, cc] = val*255
			
im_lines = morphology.binary_dilation(im_lines, morphology.disk(2))

im_lines[im_<0] = 0 
im_ = im_ - (im_lines* im_.ptp()*0.01)

im_ = im_ - np.min(im_) + (im_.ptp()*.01)

In [ ]:
out_dir = "C:/Users/eac84/Desktop/Desktop/Tasks/STL/Maps/"
name = "Colombia"
savefile(out_dir, name, im_ )

## Medellin

In [ ]:
imN = io.imread(strm_fns[5])

plt.imshow(imN[20000:20200:,3300:3700:,],cmap="jet")

## INDIAN RIVER

In [ ]:
n,s,e,w = 28,27,-80,-80.6

In [ ]:
imE = io.imread(strm_fns[5])
nsewE = n,s,e,max(-90,w)
imE = NSEW_crop1(imE, nsewE)
im = imE.copy()

im[im>0] = im[im>0] + 10
im[im<-20] = -20

plt.imshow(im, cmap="jet")

In [ ]:
plt.imshow(im, cmap="jet")

## IBERIA

In [ ]:
nsew = 44.4,34.8,5.3,-11

imW = io.imread(strm_fns[5])
imE = io.imread(strm_fns[6])

offset = np.array([90,90,-90,-90])
imW = NSEW_crop_(imW, nsew, offset)

offset = np.array([90,90,-0,-0])
imE = NSEW_crop_(imE, nsew, offset)

In [ ]:
im = np.hstack((imW,imE))
im[im<0] = im[im<0]*0.25
im[im>0] = im[im>0]+100
plt.imshow(im,cmap="jet")

In [ ]:
im_r = im.copy()
im_r[im_r<0]=0
im_r = (ndi.gaussian_filter(im_r,10) - im_r).clip(0)
im = im - 2*im_r

In [ ]:
plt.imshow(im2)

In [ ]:
im2 = proj_map_geo_to_2D(im, np.array(nsew))
im2 = im2[:,~np.any(np.isnan(im2),axis=0)]
im2 = im2 - np.min(im2) + 1

In [ ]:
scale = 0.3
sz = tuple((np.array(im2.shape)[[1,0]]*scale).astype(int))
im2 = im2 * scale
im2 = cv2.resize(im2,sz)
im2 = im2 * 0.02

In [ ]:
plt.imshow(im2, cmap="jet")

In [ ]:
out_dir = "C:/Users/eac84/Desktop/Desktop/Tasks/STL/Maps/"
name = "Ibera"
savefile(out_dir, name, im2 )


## UK

In [ ]:
n,s,e,w = 90,0,30,-30

nsewE = n,s,e,0
#imE = NSEW_crop(imE, nsewE, [90,0,90,0] )
nsewW = n,s,0,w
#imW = NSEW_crop(imW, nsewW, [90,0,0,-90])

In [ ]:
n,s = 7300, 9700
imW = imW1[n:s, 19000:]
imW[imW<0 ] = 0

imE = imE1[n:s, :700]
imE[imE<0 ] = 0

In [ ]:
imW1 = io.imread(fns[5])
imE1 = io.imread(fns[6])

In [ ]:
y = np.array([7000,9700])
x1 = np.array([19000])
x2 = np.array([700])
ntop = 90
wtop = 90

lat = -(y * 90/21600) + ntop
lng1 = (x1 * 90/21600) - wtop
lng2 = (x2 * 90/21600) - 0

nsew = np.array([*lat,*lng2,*lng1])
print(nsew)

In [ ]:
im1 = np.concatenate((imW,imE),axis=1)
im1 = proj_map_geo_to_2D(im1, nsew)

In [ ]:
im2 = im1.copy()
maxZ = np.max(im2)
im2 = im2 / maxZ
im2 = im2 ** 0.7
im2 = im2 * maxZ
im2 = im2[::-1]

In [ ]:
sz = tuple((np.array(im2.shape)[[1,0]]*0.5).astype(np.int))
im2 = im2 * 0.5
im2 = cv2.resize(im2,sz)

In [ ]:
out_dir = "C:\\Users\\eac84\\Desktop\\STL\\Maps\\"
tri = np3D.numpy2stl(im2+10)
facets = np3D.triangles_to_facets(tri)
np3D.writeSTL(facets, out_dir + "UK.stl")

AndaLucia

In [ ]:
imE1 = io.imread(fns[5])

In [ ]:
plt.imshow(imE1[12200:12800:5,-1000:-300:5])

In [ ]:
imE1.shape

## Brazil

In [ ]:
im_all = io.imread(strm_fns[1])
plt.imshow(im_all[::10,::10])

In [ ]:
def normalize(imx):

	l1,l2,l3= np.percentile(imx.ravel()[::10], [1,50,99])

	imx = imx - l2
	imx[imx<0] = imx[imx<0] / np.maximum(l1,l2-1)
	imx[imx>0] = imx[imx>0] / l3
	#imx = imx.clip(-5,5)
	return imx

In [ ]:
plt.close("all")

imx = im_all[5500:5800,10200:10700]*1.
imx = normalize(imx)
plt.figure()
plt.imshow(imx, cmap="jet")

imx = im_all[5420:5550,11150:11350]*1.
imx = normalize(imx)
plt.figure()
plt.imshow(imx, cmap="jet")


imx = im_all[5200:5800,10200:11500]*1.
imx[imx>0] = imx[imx>0]+100
imx = imx.clip(-100)
imx = normalize(imx)

plt.figure()
plt.imshow(imx, cmap="jet")

In [ ]:
plt.close("all")

imx = im_all[4000:7000:10,7500:10500:10]*1.
imx = imx.clip(-10)
imx[imx>0] = imx[imx>0]+200
imx = normalize(imx)
plt.figure()	
plt.imshow(imx, cmap="jet")

imx = im_all[5800:6500,8000:9000:]*1.
imx = normalize(imx)

plt.figure()	
plt.imshow(imx, cmap="jet")

imx = im_all[5900:6200,8300:8800:]*1.
imx = normalize(imx)

plt.figure()	
plt.imshow(imx, cmap="jet")
plt.figure()
plt.hist(imx.ravel())

## AFRICA

In [ ]:
im_4 = io.imread(strm_fns[2])
im_1 = io.imread(strm_fns[5])
im_2 = io.imread(strm_fns[6])
im_3 = io.imread(strm_fns[1])

In [ ]:
im_all = np.vstack([np.hstack([im_1,im_2]),np.hstack([im_3,im_4])])
index = [11000,31500,15500,36000]
im_0 = im_all[index[0]:index[1],index[2]:index[3]] * 1.0




In [ ]:
im_x = im_0
sz = tuple((np.array(im_x.shape)[[1,0]]*0.02).astype(int))
im_x = im_x * 0.1
im_x = cv2.resize(im_x,sz)*1.0

sz = tuple((np.array(im_x.shape)[[1,0]]*2).astype(int))
im_x = cv2.resize(im_x,sz)
plt.close("all")

plt.figure()
plt.hist(im_x.ravel()[::5],100)

x = 200
im_x[im_x>x] = 0.2*(im_x[im_x>x]-x)+x

im_x[im_x<0] = im_x[im_x<0]*.1

x = 100
im_x[im_x>x] = 0.5*(im_x[im_x>x]-x)+x

x = 50
im_x[im_x>x] = 0.5*(im_x[im_x>x]-x)+x

im_x[im_x<0] = im_x[im_x<0]*.2

x = -10
im_x[im_x<x] = 5*(im_x[im_x<x]-x)+x

im_x[im_x<0] = im_x[im_x<0] - 0.05*(im_x.max()-im_x.min())



plt.figure()
plt.hist(im_x.ravel()[::5],100)

plt.figure()
plt.imshow(im_x,cmap="jet")

plt.figure()
plt.imshow(im_x.clip(0),cmap="jet")

In [ ]:
n1,s1,e1,w1 = mat2coor( [0, 180, -90,90], [43200, 43200], index)
s1 = s1 - 180

nsew1 = (n1,s1,e1,w1)

In [ ]:

plt.figure()
sz = tuple((np.array(im_x.shape)[[1,0]]*1).astype(int))
plt.imshow(cv2.resize(im_x,sz), cmap="jet")

In [ ]:
plt.close("all")

plt.figure()
plt.hist(im_x.ravel()[::5],100)

plt.figure()
plt.imshow(im_x,cmap="jet")

In [ ]:
im_x2 = proj_map_geo_to_2D(im_x,np.array(nsew1))

plt.figure()
plt.imshow(im_x2,cmap="jet")

In [ ]:
im_x = im_x - im_x.min() + 10


out_dir = "C:/Users/eac84/OneDrive/Documents/Projects/3D Maps/Map_files/"
name = "Africa"
savefile(out_dir, name, im_x)

## Colorado

In [51]:
N,S,E,W = [41.0034002,   
           36.9925245, 
       	   -102.041585 , 
           -109.0601879]


In [52]:

target_bbox = (N, S, E, W)

result = stitch_tiles_no_rasterio(tile_files, target_bbox)

im = result.copy()

#im = proj_map_geo_to_2D(im,np.array((N, S, E, W)))
#im = im[:,~np.any(np.isnan(im),axis=0)]

plt.figure
plt.imshow(im, cmap='terrain')
plt.colorbar()
plt.show()

==== Stitching tiles without rasterio ====
Target bounding box: (41.0034002, 36.9925245, -102.041585, -109.0601879)
✅ Finished stitching without rasterio.


In [ ]:
x = 2000
im_x[im_x>x] = 0.5*(im_x[im_x>x]-x)+x

#x = 2250
#im_x[im_x>x] = 0.5*(im_x[im_x>x]-x)+x

In [ ]:
scale = 0.4
sz = tuple((np.array(im_x.shape)[[1,0]]*scale).astype(int))
im_x = im_x * scale
im_x = cv2.resize(im_x,sz)
im_x = im_x * 0.02

In [ ]:
out_dir = "C:/Users/eac84/OneDrive/Documents/Projects/3D Maps/Map_files/"
name = "Colorado"
im_x = im_x - im_x.min() + im_x.ptp()*0.1
savefile(out_dir, name, im_x)

In [ ]:
(471.5 /2) - (471/3)

## Puerto Rico

In [ ]:
nsew = 18.75, 17.65, -65.15, -67.35

In [ ]:
im_ = io.imread(strm_fns[5])
offset = np.array([90,90,-90,-90])
imE = NSEW_crop_(im_, nsew, offset).copy()

imE[imE<0] = imE[imE<0]*0.2
imE[imE>0] = imE[imE>0] + 100
plt.imshow(np.vstack([imE]), cmap="jet")

In [ ]:
im_x = imE - imE.min() + (imE.ptp()*0.1)

In [ ]:
scale = 2
sz = tuple((np.array(im_x.shape)[[1,0]]*scale).astype(int))
im_x = im_x * scale
im_x = cv2.resize(im_x,sz)
im_x = im_x * 0.02

In [ ]:
plt.imshow(im_x)

In [ ]:
out_dir = "C:/Users/eac84/OneDrive/Documents/Projects/3D Maps/Map_files/"
name = "PuertoRico"
savefile(out_dir, name, im_x)

## Keys

In [ ]:
(N, S, E, W) = 25.9,24.3, -80,-82.0


In [ ]:

target_bbox = (N, S, E, W)

result = stitch_tiles_no_rasterio(tile_files, target_bbox)


im = result.copy()
im[im<0] = im[im<0]*0.1

im = proj_map_geo_to_2D(im,np.array((N, S, E, W)))

im = im[:,~np.any(np.isnan(im),axis=0)]
#im[im<0] = im[im<0]-200

if im is not None:

	plt.figure
	plt.imshow(im, cmap='terrain')
	 
	plt.colorbar()
	plt.show()

In [ ]:

offset = np.array([90,90,-90,-90])
imE = NSEW_crop_(im_, nsew, offset).copy()*1.

imE[imE<-100] = -100
imE[imE<0] = imE[imE<0]*0.1
imE[imE>0] = imE[imE>0] + 10


In [ ]:
scale = 2
im_x = imE.copy()
sz = tuple((np.array(im_x.shape)[[1,0]]*scale).astype(int))
im_x = im_x * scale
im_x = cv2.resize(im_x,sz)
im_x = im_x * 0.02

In [ ]:
plt.imshow(im_x, cmap="jet")

In [ ]:
from scipy.ndimage import rotate

In [ ]:
DEM = im_x
rotation = 55 - 90

DEM = rotate(DEM,rotation, reshape=True, cval=-10)

DEM = DEM[480:820,:]
DEM[DEM==-10] = DEM[DEM>-10].min()
plt.figure()
plt.imshow(DEM, cmap="jet")

## Great Lakes

In [ ]:
(N, S, E, W) =  49.677, 40.465,  -74, -93.387,

In [ ]:

target_bbox = (N, S, E, W)

result = stitch_tiles_no_rasterio(tile_files, target_bbox)


im = result.copy()
im[im<0] = im[im<0]*0.1

im = proj_map_geo_to_2D(im,np.array((N, S, E, W)))

im = im[:,~np.any(np.isnan(im),axis=0)]




In [ ]:
if im is not None:

	im[0,0] = -10
	im[0,1] = 500

	plt.figure
	plt.imshow(im, cmap='jet')
	 
	plt.colorbar()
	plt.show()

In [ ]:
im_x = im.copy()

x = 500
im_x[im_x>x] = 0.5*(im_x[im_x>x]-x)+x

x = 800
im_x[im_x>x] = 0.1*(im_x[im_x>x]-x)+x

im_x = proj_map_geo_to_2D(im_x,np.array(nsew))

im_x = im_x[:,~np.any(np.isnan(im_x),axis=0)]

plt.imshow(im_x, cmap="jet")

## norway

In [ ]:
S=57.03
W=-5
N=72.07
E=33.28

In [ ]:
target_bbox = (N, S, E, W)

In [ ]:
target_bbox = (N, S, E, W)

result = stitch_tiles_no_rasterio(tile_files, target_bbox)

im = result.copy()
im[im<0] = im[im<0]*0.1

im = proj_map_geo_to_2D(im,np.array((N, S, E, W)))

im = im[:,~np.any(np.isnan(im),axis=0)]

if im is not None:
	from matplotlib import pyplot as plt
	plt.imshow(im, cmap='terrain')
	 
	plt.colorbar()
	plt.show()

## Japans

In [ ]:
S=28.75
W=118
N=55.066
E=153.69

from matplotlib import pyplot as plt
target_bbox = (N, S, E, W)

result = stitch_tiles_no_rasterio(tile_files, target_bbox)


In [ ]:

target_bbox = (N, S, E, W)

result = stitch_tiles_no_rasterio(tile_files, target_bbox)

im = result.copy()
im[im<0] = im[im<0]*0.1

im = proj_map_geo_to_2D(im,np.array((N, S, E, W)))

im = im[:,~np.any(np.isnan(im),axis=0)]
im[im<0] = im[im<0]-200

plt.imshow(im, cmap="jet")

if im is not None:

	plt.figure
	plt.imshow(im, cmap='terrain')
	 
	plt.colorbar()
	plt.show()

## mediterrian

In [ ]:
S =  29
W =  -15.
N =  47
E =  42



In [ ]:

target_bbox = (N, S, E, W)

result = stitch_tiles_no_rasterio(tile_files, target_bbox)

im = result.copy()
im[im<0] = im[im<0]*0.1

im = proj_map_geo_to_2D(im,np.array((N, S, E, W)))

im = im[:,~np.any(np.isnan(im),axis=0)]
im[im<0] = im[im<0]-500

if im is not None:

	plt.figure
	plt.imshow(im, cmap='terrain')
	 
	plt.colorbar()
	plt.show()